In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import time
import re

# 1. import list of altenative elective courses

In [2]:
import pandas as pd
df = pd.read_excel(r'UBC Okanagan MechEng Alternative Electives Names (Atypical).xlsx', header = None)
df

,0
0,COSC 301
1,COSC 315
2,COSC 322
3,COSC 406
4,MATH 319
5,MATH 340
6,MATH 350
7,PHYS 305
8,PHYS 310
9,PHYS 328


In [3]:
course_codes = list(df[0])

# 2. Use the course codes to find and scrape corresponding course names and descriptions

In [4]:
subject_links = {
    "ENGR": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=ENGR",
    "MANF": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=MANF",
    "MATH": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=8&code=MATH",
    "PHYS": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=8&code=PHYS",
    "COSC": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=8&code=COSC"
}

In [5]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  COSC 301
scraped  COSC 315
scraped  COSC 322
scraped  COSC 406
scraped  MATH 319
scraped  MATH 340
scraped  MATH 350
scraped  PHYS 305
scraped  PHYS 310
scraped  PHYS 328
scraped  PHYS 401
scraped  PHYS 402
scraped  PHYS 418
scraped  PHYS 420
scraped  PHYS 425
15


# 3. Inspect collected data and write to CSV

In [6]:
course_names

['Introduction to Data Analytics',
 'Introduction to Operating Systems',
 'Introduction to Artificial Intelligence',
 'Numerical Optimization',
 'Introduction to Partial Differential Equations',
 'Introduction to Linear Programming',
 'Complex Variables and Applications',
 'Introduction to Biophysics',
 'Introduction to Medical Physics',
 'Advanced Mechanics',
 'Electromagnetic Theory',
 'Advanced Quantum Mechanics',
 'Methods of Theoretical Physics',
 'Data and Image Processing',
 'Low-Temperature Physics']

In [7]:
course_descs

['Techniques for computation, analysis, and visualization of data using software. Manipulation of small and large data sets. Automation using scripting. Real-world applications from life sciences, physical sciences, economics, engineering, or psychology. No prior computing background is required. Credit will be granted for only one of COSC 301, DATA 301 or DATA 501. [3-2-0] Prerequisite: Either (a) third-year standing, or (b) one of COSC 111 or COSC 122 Equivalency: DATA 301.',
 'Introduction to batch, multiprogramming, and time-sharing systems. Process synchronization and communication. Main memory allocation techniques including virtual memory; process scheduling; deadlock avoidance and prevention; file organization and device management. [3-2-0] Prerequisite: All of COSC 221, COSC 222.',
 'AI and intelligent agents; state space search; game playing agents; logic and knowledge-based agents; constraint programming; planning; reasoning and decision making under uncertainty; machine lea

In [8]:
df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,COSC 301,Introduction to Data Analytics,"Techniques for computation, analysis, and visu..."
1,COSC 315,Introduction to Operating Systems,"Introduction to batch, multiprogramming, and t..."
2,COSC 322,Introduction to Artificial Intelligence,AI and intelligent agents; state space search;...
3,COSC 406,Numerical Optimization,Formulation and analysis of algorithms for con...
4,MATH 319,Introduction to Partial Differential Equations,"Methods of separation of variable, Fourier ser..."
5,MATH 340,Introduction to Linear Programming,"Linear programming problems, dual problems, th..."
6,MATH 350,Complex Variables and Applications,"Covers analytic functions, Cauchy-Riemann equa..."
7,PHYS 305,Introduction to Biophysics,Analysis of biological systems from a physicis...
8,PHYS 310,Introduction to Medical Physics,Radiation interactions with matter; use of ion...
9,PHYS 328,Advanced Mechanics,"Variational calculus, the Lagrangian Method ap..."


In [9]:
df.to_csv("UBC_Okanagan_MechEng_Alternative_Electives_Courses_(Atypical).csv", index = False)